In [1]:
import os
import keras
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Embedding, LSTM
from keras import regularizers, layers, preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf

import nltk

nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

!pip install english-words
from english_words import english_words_set

english_words_set = list( english_words_set ) 

for index in range( len( english_words_set ) ):

  english_words_set[index] = english_words_set[index].lower()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 353 kB 26.8 MB/s 
  Created wheel for english-words: filename=english_words-1.0.4-py3-none-any.whl size=352158 sha256=a9f90ded14b5e1aaea2bf1dbf745305065b43e19a66750d31803bcdc4fe06792
  Stored in directory: /root/.cache/pip/wheels/8f/7a/1f/5f54a5151359e5d80102450529a80f7a02c716e33c1cc7f46d
Successfully built english-words


In [2]:
# Obtaining the contents of the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# Importing the train & test data.

traindata = pd.read_csv('/content/drive/My Drive/NLP_Disaster/train.csv')
testdata = pd.read_csv('/content/drive/My Drive/NLP_Disaster/test.csv')

print( traindata )
print( testdata )

         id keyword  ...                                               text target
0         1     NaN  ...  Our Deeds are the Reason of this #earthquake M...      1
1         4     NaN  ...             Forest fire near La Ronge Sask. Canada      1
2         5     NaN  ...  All residents asked to 'shelter in place' are ...      1
3         6     NaN  ...  13,000 people receive #wildfires evacuation or...      1
4         7     NaN  ...  Just got sent this photo from Ruby #Alaska as ...      1
...     ...     ...  ...                                                ...    ...
7608  10869     NaN  ...  Two giant cranes holding a bridge collapse int...      1
7609  10870     NaN  ...  @aria_ahrary @TheTawniest The out of control w...      1
7610  10871     NaN  ...  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...      1
7611  10872     NaN  ...  Police investigating after an e-bike collided ...      1
7612  10873     NaN  ...  The Latest: More Homes Razed by Northern Calif...      1

[76

In [ ]:
traindata.fillna('').astype(str).apply(lambda x:x.str.len()).max()

id            5
keyword      21
location     49
text        157
target        1
dtype: int64

# Lemmatizing and removing junk like URLs.

In [ ]:
list_traincp = list( traincp['text'] )

for index in range( len( list_traincp ) ):

  new_sentence = ""

  for word in ( list_traincp[ index ] ).split(" "):

    if lemmatizer.lemmatize( word.lower() ) in english_words_set:

      new_sentence += " " + word

    elif lemmatizer.lemmatize( word.lower()[ 1: ] ) in english_words_set:

      new_sentence += " " + word[ 1: ]

  list_traincp[ index ] = new_sentence

NameError: ignored

# Function to Lemmatize and remove stuff like URLs from the text.

In [4]:

def lemmabin( List ):

  #list_traincp = list( traincp['text'] )

  for index in range( len( List ) ):

    new_sentence = ""

    for word in ( List[ index ] ).split(" "):

      if lemmatizer.lemmatize( word.lower() ) in english_words_set:

        new_sentence += " " + word

      elif lemmatizer.lemmatize( word.lower()[ 1: ] ) in english_words_set:

        new_sentence += " " + word[ 1: ]

    List[ index ] = new_sentence


  return List



## Deprecated code:

In [ ]:
# Going through the texts and finding all the real words within the text.

list_traincp = list( traincp['text'] )

for index in range( len( list_traincp ) ):

  new_sentence = ""

  for word in ( list_traincp[ index ] ).split(" "):

    if lemmatizer.lemmatize( word.lower() ) in english_words_set:

      new_sentence += " " + word

    elif lemmatizer.lemmatize( word.lower()[ 1: ] ) in english_words_set:

      new_sentence += " " + word[ 1: ]

  list_traincp[ index ] = new_sentence

# Making a tokenizer and converting the text to sequences of numbers with the tokenizer.

* I need to tokenize the text on the keyword too.

In [5]:
# Creating copies to work with.
traincp = traindata.copy()
testcp = testdata.copy()


# Creating a tokenizer object.
tokenizer = Tokenizer()

lemmad_traincp = lemmabin( list( traincp['text'] ) )

lemmad_testcp = lemmabin( list( testcp['text'] ) )


# Fitting the tokenizer on the text.
# Updates the internal vocabulary of the tokenizer.
# **This needs to be done using both the train and test data**.
tokenizer.fit_on_texts( lemmad_traincp )

word_index = tokenizer.word_index

# Converting the texts to sequences of.
train_sequences = tokenizer.texts_to_sequences( lemmad_traincp )

test_sequences = tokenizer.texts_to_sequences( lemmad_testcp )

Xpad_train = pad_sequences( train_sequences, )

Xpad_test = pad_sequences( test_sequences, )


In [6]:
print( word_index )

{'the': 1, 'a': 2, 'in': 3, 'to': 4, 'of': 5, 'and': 6, 'i': 7, 'is': 8, 'for': 9, 'on': 10, 'you': 11, 'my': 12, 'with': 13, 'that': 14, 'at': 15, 'by': 16, 'it': 17, 'this': 18, 'from': 19, 'are': 20, 'be': 21, 'was': 22, 'have': 23, 'like': 24, 'as': 25, 'so': 26, 'just': 27, 'but': 28, 'up': 29, 'your': 30, 'not': 31, 'me': 32, 'will': 33, 'has': 34, 'all': 35, 'an': 36, "i'm": 37, 'if': 38, 'when': 39, 'no': 40, 'out': 41, 'we': 42, 'get': 43, 'via': 44, 'new': 45, 'fire': 46, 'about': 47, 'more': 48, 'or': 49, 'he': 50, 'they': 51, 'been': 52, 'what': 53, 'how': 54, 'over': 55, 'one': 56, 'people': 57, 'who': 58, 'now': 59, "t's": 60, 'into': 61, "don't": 62, 'do': 63, 'can': 64, 'there': 65, 'than': 66, 'would': 67, 'some': 68, 'still': 69, 'his': 70, 'police': 71, 'her': 72, 'body': 73, 'video': 74, 'suicide': 75, 'got': 76, 'why': 77, 'had': 78, 'off': 79, 'back': 80, 'us': 81, 'were': 82, 'first': 83, 'know': 84, 'nuclear': 85, 'see': 86, 'our': 87, 'hit': 88, 'man': 89, 'lov

# Separating out the labels

In [ ]:
print( testcp )

         id keyword location                                               text
0         0     NaN      NaN                 Just happened a terrible car crash
1         2     NaN      NaN  Heard about #earthquake is different cities, s...
2         3     NaN      NaN  there is a forest fire at spot pond, geese are...
3         9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4        11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan
...     ...     ...      ...                                                ...
3258  10861     NaN      NaN  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259  10865     NaN      NaN  Storm in RI worse than last hurricane. My city...
3260  10868     NaN      NaN  Green Line derailment in Chicago http://t.co/U...
3261  10874     NaN      NaN  MEG issues Hazardous Weather Outlook (HWO) htt...
3262  10875     NaN      NaN  #CityofCalgary has activated its Municipal Eme...

[3263 rows x 4 columns]


In [7]:
train_labels = np.asarray( traincp['target'] )
#test_labels = np.array( testcp['target'] )

In [ ]:
print( lemmad_traincp )

[' Our Deeds are the Reason of this earthquake May ALLAH Forgive us all', ' Forest fire near La Canada', ' All residents to shelter in are being by No other or shelter in place orders are', ' people receive wildfires orders in California', ' Just got sent this photo from Ruby Alaska as smoke from wildfires into a school', ' Update California in both due to Lake County fire wildfires', ' flood Heavy rain causes flash of streets in Colorado Springs areas', " I'm on top of the hill and I can see a fire in the", ' an now in the across the street', " I'm afraid that the tornado is to our", ' Three people from the heat wave so far', ' South Tampa is WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I DO WHAT AM I DO', " Florida Tampa or I've lost count", ' Flood in ago We ago', ' Damage to school bus on in multi car crash', ' up', ' I love fruits', ' Summer is', ' My car is so fast', ' What a', ' this is', ' London is cool', ' Love', ' What a wonderful', '', " No can't eat that hit", ' Was in NYC 

# Creating a train & valid variables from the padded sequences.

In [ ]:
print( Xpad_train )
print( Xpad_test )

[[   0    0    0 ... 2650   81   35]
 [   0    0    0 ...  182  538 1157]
 [   0    0    0 ...  539  939   20]
 ...
 [   0    0    0 ...  185    5  415]
 [   0    0    0 ...    3  217  947]
 [   0    0    0 ...   91  168  512]]
[[   0    0    0 ... 1271  108  103]
 [   0    0    0 ...  863  505 2018]
 [   0    0    0 ...  255  109   35]
 ...
 [   0    0    0 ...  529    3  941]
 [   0    0    0 ...  400  325  220]
 [   0    0    0 ...   34  123 1078]]


In [8]:

x_train = Xpad_train
y_train = train_labels

x_test = Xpad_test


In [ ]:
Xpad_train[ 1000 ]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0, 667, 989,  73, 409,
       707,  10,  14], dtype=int32)

In [ ]:


print( Xpad_train.shape )

print( Xpad_test.shape )


(7613, 29)
(3263, 26)


In [17]:


#print( len( word_index ) )
print( Xpad_train.shape )



(7613, 29)


Need to look through this model and create a bunch of copies to see 

In [20]:

model = Sequential()

model.add( Embedding( input_dim = ( len( word_index ) + 1 ), output_dim = 10, input_length = Xpad_train.shape[ 1 ]) )
model.add( layers.Conv1D(128, 27, activation='relu') )
model.add( layers.MaxPooling1D(9, padding="same") )
model.add( layers.Dropout(0.5) )
model.add( layers.Conv1D(128, 1, activation='relu') )
model.add( layers.Dropout(0.5) )
model.add( layers.Bidirectional(layers.GRU(32, dropout=0.2, recurrent_dropout=0.2)) )
model.add( Dense(1, activation='sigmoid') )
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 29, 10)            61390     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 3, 128)            34688     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1, 128)            16512     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)               

## Another NN

In [ ]:




model = Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(29,)))
model.add(layers.Dense(32, activation='relu')),
model.add(layers.Dense(32, activation='relu')),
model.add(layers.Dense(32, activation='relu')),
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])






## Running the code

In [21]:


history = model.fit( x_train,
                    y_train,
                    epochs=25,
                    batch_size=128,
                    )



Epoch 1/25
60/60 [==============================] - 5s 12ms/step - loss: 0.6828 - acc: 0.5676
Epoch 2/25
60/60 [==============================] - 1s 12ms/step - loss: 0.5821 - acc: 0.6900
Epoch 3/25
60/60 [==============================] - 1s 12ms/step - loss: 0.3988 - acc: 0.8338
Epoch 4/25
60/60 [==============================] - 1s 12ms/step - loss: 0.2956 - acc: 0.8832
Epoch 5/25
60/60 [==============================] - 1s 12ms/step - loss: 0.2187 - acc: 0.9201
Epoch 6/25
60/60 [==============================] - 1s 12ms/step - loss: 0.1677 - acc: 0.9398
Epoch 7/25
60/60 [==============================] - 1s 12ms/step - loss: 0.1424 - acc: 0.9498
Epoch 8/25
60/60 [==============================] - 1s 12ms/step - loss: 0.1187 - acc: 0.9574
Epoch 9/25
60/60 [==============================] - 1s 12ms/step - loss: 0.1034 - acc: 0.9631
Epoch 10/25
60/60 [==============================] - 1s 12ms/step - loss: 0.0975 - acc: 0.9632
Epoch 11/25
60/60 [==============================] - 1s 12m